# CS 182 Fall 2020 Problem set 0 - Programming

This is the programming component of Problem Set Zero for CS 182. We provide this for reference, and so students get a sense of the Python background needed.
This problem set will be due in the first week of classes. We will provide later more information on how to work on and submit this problem set.


### Question 1 
In this question you will implement a _priority queue_ this will be a class `PQueue` which has the following interface:

```
queue = PQueue() # create new queue
queue.push(item,priority) # insert item into the queue with the priority
queue.pop() # pop from the queue the item with minimum priority and return both item and priority
```

This is not an algorithms course, and so it's OK if you don't use the most efficient implementation for this question.
You should not import any Python library or module in your solution to this problem.

In [149]:
class PQueue:
    # initialise queue as list
    def __init__(self):
        self.queue = []
    
    # insert item into the queue with the priority
    def push(self, item, priority):

        # if queue is empty then append item
        if len(self.queue) < 1:
            self.queue.append([item, priority])
            return
        else:
        # check if same item is in queue
          for i in range(len(self.queue)):
            if self.queue[i][0] == item:
                if priority < self.queue[i][1]:
                  del self.queue[i]
                else:
                  return
                break

          # insert item into queue according to priority
          for j in range(len(self.queue)):
                if priority < self.queue[j][1]:
                    self.queue.insert(i, [item, priority])
                    return
                else:
                    if i == len(self.queue) - 1:
                        self.queue.append([item, priority])
                        return 

      
    # pop from the queue the item with minimum priority and return both item and priority
    def pop(self):
        if len(self.queue) > 0:
            item = self.queue.pop(0)
            return item

    # calculate length of the queue
    def len(self):
        return (len(self.queue))

In [150]:
queue = PQueue() # create new queue
queue.push(1,1) # insert item into the queue with the priority
queue.push(1,2)
queue.pop() # pop from the queue the item with minimum priority and return both item and priority


[1, 1]

### Question 2

In this question you should implement a "breadth first search" to find the minimum distance between two vertices in a graph.

We will represent in this question a graph using the _adjacency matrix_ representation. 

We represent an $n$ vertex weighted graph $G$ by a list `G` of $n$ lists, each of length $n$, such that for every $i,j  \in \{0,\ldots, n\}$,
`G[i][j]` is the (non negative) length  $G_{i,j}$ of the edge from vertex $i$ and vertex $j$. We use `G[i][j] = float("inf")` if there is no such edge. 

Write a function `distance(G,i,j)` that returns the distance between $i$ and $j$ which is defined as the minimum of the sum
$G_{i,k_1}+G_{k_1,k_2}+\cdots,+G_{k_{\ell-1},k_\ell} + G_{k_\ell,j}$ over all possible choices $0 \leq \ell \leq n-2$ 
and intermediate vertices $k_1,\ldots,k_\ell$.




In [151]:
# fill in the code below 
def distance(G,i,j):
  
   def bfs(G, start):
      # initialize distance travelled and vertices visited as dicts
      distance = []
      visited = []
      # call PQueue
      Q = PQueue()

      # initialize distance travelled and vertices dicts
      for vertex in range(len(G)):

         # initialise visited dict to 'none'
         visited.append(None)

         # if there is no edge, append "inf"
         if vertex != start:
            distance.append(float("inf"))
         
         # insert first vertex as 'start' and initialise distance to 0
         else:
            distance.insert(start, 0)
      
      # Add the vertex to start the bfs to the empty queue, where priority == distance
      for vertex in range(len(G)):
         Q.push(vertex, distance[vertex])

      # MAIN LOOP (repeat until queue is empty)
      while Q.len() != 0:
         # extract vertex from the queue
         location = Q.pop()[0]

         # cycle through vertices
         for vertex in range(len(G[location])):
           
            # if there is no edge (marked by "inf")
            if G[location][vertex] != float("inf"):
               # store location + distance in temp variable
               temp = G[location][vertex] + distance[location]
               
               if temp < distance[vertex]:
                  distance[vertex] = temp
                  # Mark that vertex visited
                  visited[vertex] = location
                  # add that vertex's neighbors to the queue 
                  Q.push(vertex, temp)

      return distance

   return(bfs(G,i)[j])

In [152]:
# For student testing
null = float("inf")

G = [ [null, 1, null, null, null],
      [1, null, null, null, 1],
      [null, null, null, null, 1],
      [null, null, null, null, 6],
      [null, 1, 5, 1, null],
      [null, null, null, 1, 1],
      [1, 1, 1, null, null],
      [null, null, null, null, null],
      [1, null, null, 4, null],
      [null, 3, null, null, 1] ]

print(distance(G, 0, 0))
print(distance(G, 0, 2))
print(distance(G, 0, 4))
print(distance(G, 0, 1))

0
7
2
1


## Question 3

In this question we'll use the code above to figure out the time to travel between different T station. 


Below we supply code that will grab for you two files:

* `HRTravelTimesApr-June2019.csv` - travel times for trips between April to June 2019 between two stations on the same line (in comma separated format). The stations have a numeric "station ID"

* `stops.json`  - a file which maps the station IDs into information about stations, including their names. (There can be several IDs with the same name, since ID correspond to platforms)

We provide for you code that reads these files and generates:

* `distances` - a [pandas](https://pandas.pydata.org/) dataframe which can be thought of as a two dimensional array that rather than being indexed by integers $0,\ldots,n-1$ is indexed by station IDs.
  `distances[id1][id2]` is the average time it took to travel from `id1` to `id2` if there are on the same T line, and `float("nan")` (not a number) if they are not on the same line. 
  (If you are not familiar with pandas then look it up or ask us for resources. You won't need much for this exercise but it's a very useful framework to know about.)

* `stopnames` a Python Dictionary such that  `stopnames[id]` is the name of `id`

Your goal is to write:

1. Function `get_names(distances,stopnames)` that returns a list of the unique names corresponding to all id's in `distances`. 
  (Hint: you can use `list(distances.index.values)` and `list(distances.columns.values)` to get the lists of rows and column identifiers)

2. If `names` is list returned by `get_names(distances,stopnames)`, write a function `gen_graph(distances,stopnames)` (using `get_names`) that returns an `n` vertex
graph `G` on `n` vertices such that the distance between `i` and `j` is the minimum average time to travel between all `id1` and `id2` such that `names[i]=stopnames[id1]` and `names[j]=stopnames[id2]`
if this average is not a number then we set it as `float("inf")` (you can use `math.isnan(num)` to test if `num` is a number).

3. Use `distance` on this graph to return the time it takes to go from Harvard to the airport.


## Provided Code

In [153]:
# Get the files from the MBTA websites
!wget https://www.arcgis.com/sharing/rest/content/items/a70a2151c74f4ea7b6688970b76d2ff7/data -O mbta_times.zip
!unzip -o mbta_times.zip
!wget https://api-v3.mbta.com/stops -O stops.json

--2020-12-09 23:37:08--  https://www.arcgis.com/sharing/rest/content/items/a70a2151c74f4ea7b6688970b76d2ff7/data
Resolving www.arcgis.com (www.arcgis.com)... 18.234.22.251, 34.199.206.244, 34.233.149.104, ...
Connecting to www.arcgis.com (www.arcgis.com)|18.234.22.251|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ago-item-storage.s3.us-east-1.amazonaws.com/a70a2151c74f4ea7b6688970b76d2ff7/Travel_Times_2019.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEM%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJIMEYCIQD%2FMK%2BqYd%2F4HQvCGFFOX7VHqRNuKcYh%2BJDDip82Tb3OVgIhAPTdIhQE%2BHXMBO9Hbh7EbpSYEUdUjOqdYmTwAWBA9yfDKrQDCGgQABoMNjA0NzU4MTAyNjY1IgxGrsXKLLGt8Mr56RMqkQNp50U3CjnRqaCv3AVG8SK1P1V9OLmmYkz7GjBEq%2FqvYol18VqvLmq46cteEshtNqGd9kULxnCEiQI90QA8d8Ht4gYXh17sh78n7TpYsV%2Fsyi50hVyBz1AotmFOHdZn1FHxZT5zV%2BAxQel%2Bro96l2mcRCGWimlnV303WrWQd6wkErZK6%2BkOolPZ4ksaE3uYbCU%2FbDIfBoC8QddaG8xgcpQVMXxnNTa0skZq0s3RkqkgWidCxKnnQnodsWoz9XFVdoQj1Lv%2Fb1o9nG%2Bd5u5OaNgL2Fx0

In [154]:
# make distances a table of all average times from one station to the other on the same line
import pandas as pd
times = pd.read_csv('HRTravelTimesApr-June2019.csv', low_memory = False)
times = times.astype({'from_stop_id': str, 'to_stop_id': str })
distances = times.groupby(['from_stop_id','to_stop_id'])['travel_time_sec'].mean().unstack()

In [172]:
# Load the name of all stations into a dictionary
import json
with open('stops.json') as json_file: 
    stops = json.load(json_file)["data"]
stopnames = {d["id"] : d["attributes"]["name"] for d in stops}

{'17354': 'Lynnfield St @ St Mary Cemetery', '1339': 'Walnut Ave @ St Richard St', '82851': 'Walnut St @ Page Rd', 'node-129-lobby': 'Forest Hills', '9221': 'Main St @ Banks Pl', '4001': 'S Franklin St opp Hillsdale Rd', '3253': 'Sea St @ Babcock St', 'door-knncl-sbmain': 'Kendall/MIT - Main St, Buses, Cambridge Center', 'Rockport-S': 'Rockport', '89608': 'Totten Pond Rd opp Craig Ln', 'node-armnl-wescstairs-lobby': 'Arlington', '111146': 'Pemberton Point', '9172': '116 Riverside Ave', '8553': 'Sawmill Brook Pkwy @ Keller Path', '4614': 'Endicott St @ Market Basket', 'node-935-platform': 'World Trade Center', '70238': 'Cleveland Circle', '6250': 'Malden St @ Stark Ave', '1804': 'Brookline Ave @ Longwood Ave', '5037': 'George St @ Winthrop St', '8465': 'South Rd @ Crescent Ave', '70007': 'Jackson Square', '6177': 'Lafayette St @ Saltonstall Pkwy', '5598': '296 Washington Ave opp Jefferson Ave', 'node-haecl-congogplatn-passage': 'Haymarket', 'door-prmnl-prudential': 'Prudential - Prudent

### Your code here

In [156]:
def get_names(distances, stopnames):
    
    # initialize names as set (so no duplicates)
    names = set()
    # initialise row and column lists of rows and column identifiers
    row = list(distances.index.values)
    column = list(distances.columns.values)

    # iterate through rows to populate table
    for i in range(len(row)):
      names.add(stopnames[row[i]])
      names.add(stopnames[column[i]])
    # returns a list of the unique names corresponding to all id's in distances
    return list(names)
    

In [177]:
# for student testing
names = get_names(distances, stopnames)
#print(names)

['Wood Island', 'Bowdoin', 'Forest Hills', 'JFK/UMass', 'Quincy Center', 'Back Bay', 'Andrew', 'Wonderland', 'Malden Center', 'Assembly', 'Government Center', 'Central', 'Suffolk Downs', 'Downtown Crossing', 'Massachusetts Avenue', 'Kendall/MIT', 'State', 'Maverick', 'Ashmont', 'Haymarket', 'Stony Brook', 'Harvard', 'Savin Hill', 'Jackson Square', 'Fields Corner', 'Orient Heights', 'Braintree', 'Oak Grove', 'Ruggles', 'Revere Beach', 'Shawmut', 'Community College', 'Airport', 'Chinatown', 'Aquarium', 'Charles/MGH', 'North Station', 'South Station', 'North Quincy', 'Wellington', 'Roxbury Crossing', 'Beachmont', 'Park Street', 'Quincy Adams', 'Broadway', 'Tufts Medical Center', 'Sullivan Square', 'Alewife', 'Green Street', 'Davis', 'Porter']


In [158]:
import math

def gen_graph(distances, stopnames, names):
    
    # declare G as empty list and initialise to null value "inf"
    G = []
    for i in range(len(names)):
        G.append([])
        for j in range(len(names)):
            G[i].append(float("inf"))

    # as above, initialise rows and columns to values in distances 
    row = list(distances.index.values)
    column = list(distances.columns.values)
  
    # populate graph with distances
    for id1 in row:
        for id2 in column:
            i = names.index(stopnames[id1])
            j = names.index(stopnames[id2])
            if not (math.isnan(distances.loc[id1].loc[id2])):
                G[i][j] = min(distances.loc[id1].loc[id2], G[i][j])
    # returns an n vertex graph G on n vertices 
    return G
  

In [174]:
# for student testing
G = gen_graph(distances, stopnames, names)

[[inf, 660.1792452830189, inf, inf, inf, inf, inf, 580.2088535754824, inf, inf, 560.0030555194382, inf, 224.95432387312187, inf, inf, inf, 465.9857681310112, 232.3310810810811, inf, inf, inf, inf, inf, inf, inf, 123.85497301978549, inf, inf, inf, 442.94858783467987, inf, inf, 59.14780465659251, inf, 381.8704872178495, inf, inf, inf, inf, inf, inf, 316.37410791702797, inf, inf, inf, inf, inf, inf, inf, inf, inf], [602.6909809084924, inf, inf, inf, inf, inf, inf, 1218.5040661334767, inf, inf, 16.773960928281106, inf, 864.9170813718897, inf, inf, inf, 109.07998950957251, 357.67438582360046, inf, inf, inf, inf, inf, inf, inf, 764.4010738255033, inf, inf, inf, 1081.9864892115345, inf, inf, 495.089388050853, inf, 197.81005512386267, inf, inf, inf, inf, inf, inf, 956.2149909329034, inf, inf, inf, inf, inf, inf, inf, inf, inf], [inf, inf, inf, inf, inf, 764.3198239348891, inf, inf, 2210.8426644182123, 1800.9841150405484, inf, inf, inf, 1083.7274008956708, 629.2892026578073, inf, 1190.946047678

In [183]:
#find index of locations
for i in range(len(names)):
     if names[i] == "Harvard":
         Harvard = i
     elif names[i] == "Airport":
         Airport = i

distance_in_hours = distance(G, Harvard, Airport)
distance_in_minutes = distance_in_hours/60
print(distance_in_minutes)

11.657537805417045
